In [ ]:
from unsloth import FastLanguageModel
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import torch

def load_model():
    model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
    # unsloth/qwen2-1.5b-bnb-4bit
    # unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit
    max_seq_length = 2048

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_id,
        max_seq_length=max_seq_length,
        dtype=torch.float16,
        load_in_4bit=True,
    )

    model.eval()
    tokenizer.padding_side = "right"

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    return HuggingFacePipeline(pipeline=pipe)


def build_prompt():
    template = """
You are a professional career advice AI chatbot, specializing in helping undergraduate and postgraduate students with their career questions.  
Provide accurate, encouraging, and practical advice based on the student’s question.

Here are some examples of how you should respond:

Student: I’m about to graduate and don’t know how to write a resume. Can you help?  
Chatbot: Of course! A good resume highlights your education, internships, and skills clearly. Keep it concise—usually one page is best. I can suggest some resume templates or connect you with a career advisor for more personalized help.

Student: I’m not sure what career suits me. Any advice?  
Chatbot: That’s a common concern. You might want to try some career interest assessments to learn more about your strengths and preferences. Attending career talks and workshops can also help. If you want, I can help you book a session with a career counselor for deeper guidance.

Student: Where can I find internship opportunities?  
Chatbot: Great question! Popular internship websites include LinkedIn, Indeed, and your university’s career portal. Also, consider attending job fairs and networking with alumni. I can help you create an internship application plan if you'd like.

Student: I feel anxious about my future and don’t know what to do.  
Chatbot: It’s completely normal to feel this way. Try some relaxation techniques and talk to friends or mentors about your concerns. If you need more support, I recommend reaching out to your university’s counseling or career services.

Student: Can you tell me about the hiring process at a specific company?  
Chatbot: Detailed info about specific company hiring processes can be complex. I suggest contacting your university’s career center or your tutor for personalized advice. Meanwhile, I can help find publicly available info if you want.

Now, please answer the following question accordingly:

Student: {question}
Chatbot:"""
    return PromptTemplate.from_template(template)


def create_chain(llm, prompt_template):
    return LLMChain(llm=llm, prompt=prompt_template)


def main():
    print(" Loading model, please wait...")
    llm = load_model()

    print(" Model loaded. Building chat chain...")
    prompt_template = build_prompt()
    qa_chain = create_chain(llm, prompt_template)

    print(" Career Advice Chatbot is ready! Enter your questions (type 'exit' to quit).")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print(" Goodbye! Wish you success in your career!")
            break

        # Use invoke() to get output only, avoid printing the whole prompt
        raw = qa_chain.invoke({"question": user_input})["text"]
        answer = raw.split("Chatbot:")[-1].strip()
        print(f"\nAI: {answer}")



if __name__ == "__main__":
    main()


In [ ]:
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import faiss
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
import json

# Load test text data
with open("test_texts.pkl", "rb") as f:
    test_texts = pickle.load(f)

# Initialize embedding model (same as used in QA)
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Load FAISS index and original text library (for retrieval context)
index = faiss.read_index("qa_index.faiss")
with open("qa_chunks.pkl", "rb") as f:
    texts = pickle.load(f)

# Load local Qwen2 model (4bit)
model_name = "unsloth/qwen2-1.5b-bnb-4bit"
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True
)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"

def answer_question(query, top_k=3):
    # 4.1 Encode query
    query_vec = embedder.encode([query], convert_to_numpy=True)
    
    # 4.2 Search top_k relevant chunks
    D, I = index.search(query_vec, top_k)
    
    # 4.3 Get retrieved texts
    retrieved_chunks = [texts[i] for i in I[0]]
    
    # 4.4 Join context
    context = "\n---\n".join(retrieved_chunks)
    
    # 4.5 Construct prompt in English
    prompt = f"""You are an intelligent QA assistant. Please answer the user's question based on the following background knowledge:
Background documents:
{context}
Here are some examples of how you should respond:
Student: I'm about to graduate and don't know how to write a resume. Can you help?  
Chatbot: Of course! A good resume highlights your education, internships, and skills clearly. Keep it concise—usually one page is best. I can suggest some resume templates or connect you with a career advisor for more personalized help.

Student: I'm not sure what career suits me. Any advice?  
Chatbot: That's a common concern. You might want to try some career interest assessments to learn more about your strengths and preferences. Attending career talks and workshops can also help. If you want, I can help you book a session with a career counselor for deeper guidance.

Student: I feel anxious about my future and don't know what to do.  
Chatbot: It's completely normal to feel this way. Try some relaxation techniques and talk to friends or mentors about your concerns. If you need more support, I recommend reaching out to your university's counseling or career services.

Student: Can you tell me about the hiring process at a specific company?  
Chatbot: Detailed info about specific company hiring processes can be complex. I suggest contacting your university's career center or your tutor for personalized advice. Meanwhile, I can help find publicly available info if you want.

Student: Where can I find internship opportunities?  
Chatbot: Great question! Popular internship websites include LinkedIn, Indeed, and your university's career portal. Also, consider attending job fairs and networking with alumni. I can help you create an internship application plan if you'd like.

Now, please answer the following question accordingly:
Student: {query}
Chatbot:"""
    # 4.6 Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 4.7 Generate answer with sampling for diversity
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    # 4.8 Decode output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 4.9 Extract answer only
    answer = generated_text[len(prompt):].strip()
    return answer

# Extract questions and reference answers from Q&A text chunks (supports multi-line answers)
def extract_qa(text):
    q_match = re.search(r"Q:\s*(.*)", text)
    a_match = re.search(r"A:\s*(.*)", text, re.DOTALL)
    question = q_match.group(1).strip() if q_match else ""
    answer = a_match.group(1).strip() if a_match else ""
    return question, answer

def evaluate_semantic_similarity(test_texts, max_samples=200):
    """
    Calculate semantic similarity evaluation
    Returns: average similarity, similarity list, detailed results
    """
    total = min(len(test_texts), max_samples)
    similarities = []
    detailed_results = []
    
    print(f" Starting semantic similarity evaluation (Total {total} samples)...")
    
    for i, text in enumerate(test_texts[:total]):
        question, true_answer = extract_qa(text)
        if not question or not true_answer:
            print(f"Skipping sample {i+1}, missing question or answer.")
            continue
        
        # Generate model answer
        pred_answer = answer_question(question)
        if not pred_answer:
            print(f"Warning: Empty prediction for query: {question}")
            pred_answer = ""
        
        # Calculate semantic similarity
        try:
            true_vec = embedder.encode([true_answer], convert_to_numpy=True)
            pred_vec = embedder.encode([pred_answer], convert_to_numpy=True)
            sim = cosine_similarity(true_vec, pred_vec)[0][0]
        except Exception as e:
            print(f"Error calculating similarity for sample {i+1}: {e}")
            sim = 0.0
        
        similarities.append(sim)
        
        # Save detailed results
        detailed_results.append({
            "index": i + 1,
            "question": question,
            "reference_answer": true_answer,
            "generated_answer": pred_answer,
            "similarity_score": float(sim)
        })
        
        print(f"Sample {i+1}/{total}")
        print(f"Question: {question[:80]}...")
        print(f"Reference answer: {true_answer[:80]}...")
        print(f"Generated answer: {pred_answer[:80]}...")
        print(f"Similarity: {sim:.4f}")
        print("-" * 50)
    
    # Calculate statistical metrics
    similarities = np.array(similarities)
    
    results = {
        "total_questions": len(similarities),
        "mean_similarity": float(np.mean(similarities)),
        "std_similarity": float(np.std(similarities)),
        "median_similarity": float(np.median(similarities)),
        "min_similarity": float(np.min(similarities)),
        "max_similarity": float(np.max(similarities)),
        "detailed_results": detailed_results
    }
    
    return results, similarities

def print_evaluation_summary(results, similarities):
    """Print evaluation results summary"""
    print("\n" + "="*60)
    print(" Semantic Similarity Evaluation Results")
    print("="*60)
    print(f"Total test samples: {results['total_questions']}")
    print(f"Mean semantic similarity: {results['mean_similarity']:.4f}")
    print(f"Standard deviation: {results['std_similarity']:.4f}")
    print(f"Median: {results['median_similarity']:.4f}")
    print(f"Minimum: {results['min_similarity']:.4f}")
    print(f"Maximum: {results['max_similarity']:.4f}")
    
    # Similarity distribution
    print(f"\n Similarity Distribution:")
    print(f"[0.8-1.0] (Excellent): {np.sum(similarities >= 0.8)} ({np.mean(similarities >= 0.8)*100:.1f}%)")
    print(f"[0.6-0.8) (Good): {np.sum((similarities >= 0.6) & (similarities < 0.8))} ({np.mean((similarities >= 0.6) & (similarities < 0.8))*100:.1f}%)")
    print(f"[0.4-0.6) (Fair): {np.sum((similarities >= 0.4) & (similarities < 0.6))} ({np.mean((similarities >= 0.4) & (similarities < 0.6))*100:.1f}%)")
    print(f"[0.0-0.4) (Poor): {np.sum(similarities < 0.4)} ({np.mean(similarities < 0.4)*100:.1f}%)")

def show_examples(results, num_examples=3):
    """Show best and worst examples"""
    detailed = results["detailed_results"]
    sorted_results = sorted(detailed, key=lambda x: x["similarity_score"], reverse=True)
    
    print(f"\n Top {num_examples} highest similarity samples:")
    for i, item in enumerate(sorted_results[:num_examples]):
        print(f"\n--- Sample {i+1} (Similarity: {item['similarity_score']:.4f}) ---")
        print(f"Question: {item['question']}")
        print(f"Reference answer: {item['reference_answer'][:100]}...")
        print(f"Generated answer: {item['generated_answer'][:100]}...")
    
    print(f"\n Bottom {num_examples} lowest similarity samples:")
    for i, item in enumerate(sorted_results[-num_examples:]):
        print(f"\n--- Sample {i+1} (Similarity: {item['similarity_score']:.4f}) ---")
        print(f"Question: {item['question']}")
        print(f"Reference answer: {item['reference_answer'][:100]}...")
        print(f"Generated answer: {item['generated_answer'][:100]}...")

if __name__ == "__main__":
    # Run semantic similarity evaluation
    results, similarities = evaluate_semantic_similarity(test_texts, max_samples=200)
    
    # Print statistical summary
    print_evaluation_summary(results, similarities)
    
    # Show examples
    show_examples(results)
    
    # Save detailed results to file
    with open("semantic_similarity_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"\n Detailed results saved to 'semantic_similarity_results.json'")
    print(f" Evaluation completed! Mean semantic similarity: {results['mean_similarity']:.4f}")